In [2]:
import requests
import time
from tqdm import tqdm

SYSTEM_PROMPT = (
    "Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы."
)
MODEL_NAME = "t-tech/T-lite-it-1.0"

USER_PROMPT_MAIN = '''
Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки.

Запрос пользователя: {query}
Ответ:
'''

USER_PROMPT_POEM = '''
Ты — поэт-ассистент, вдохновляющийся классической русской поэзией.
На основе запроса пользователя сочини **новое** стихотворение в духе русской поэтической традиции.
Можешь использовать метафоры, эпитеты, ритм, рифму — всё, что придаёт тексту поэтичность.
Если знаешь строки из известных произведений — можешь вдохновляться ими, но **не копируй** дословно и не цитируй.
**НЕ ИСПОЛЬЗУЙ** в стихотворении имена и фамилии авторов.

Запрос пользователя: {query}
Стихотворение:
'''

def get_response_rag(query, is_poem):
    if is_poem:
        url = "http://localhost:8001/api/poem_async"
    else:
        url = "http://localhost:8001/api/generate_async"
    res = requests.post(
        url,
        json={"query": query},
        timeout=60
    )
    task_id = res.json()["task_id"]

    for _ in range(10):
        time.sleep(2)
        res = requests.get(f"http://localhost:8001/api/result/{task_id}")
        data = res.json()
        if data.get("status") == "completed":
            return data["result"]["response"]
    else:
        raise TimeoutError('ERROR')


In [23]:
gen_queries = [
    'Напиши стихотворение в стиле Александра Пушкина. Тема - зима в деревне',
    'Придумай стих в стиле автора Владимира Маяковского на тему советского патриотизма',
    'Напиши стихотворение на тему осеннего дождливого дня как поэт Осип Мандельштам',
    'Сгенерируй стих на тему безответной любви в автора Анны Ахматовой',
    'Придумай стихотворение на тему одиночества в городе. Пиши в стиле автора Иосифа Бродского'
]

In [4]:
# q = gen_queries[0]
# resp = get_response_rag(q, is_poem=True)

In [5]:
gen_response_rag_lst_p = []
for i in tqdm(range(3)):
    q = gen_queries[0]
    resp = get_response_rag(q, is_poem=True)
    gen_response_rag_lst_p.append(resp)

100%|██████████| 3/3 [00:40<00:00, 13.47s/it]


In [ ]:
import pandas as pd 

gen_authors_lst = ['Александр Пушкин']
gen_themes_lst = ['Зима в деревне']

pd.DataFrame({
    'author': gen_authors_lst,
    'theme': gen_themes_lst,
    'queries': gen_queries[0],
    'LLM_RAG_LORA_P1': [gen_response_rag_lst_p[0]],
    'LLM_RAG_LORA_P2': [gen_response_rag_lst_p[1]],
    'LLM_RAG_LORA_P3': [gen_response_rag_lst_p[2]]
})#.to_csv('../data/ResponsesGen_llm_rag_loraP.csv', index=False)

In [ ]:
# gen_response_rag_lst = []
# for q in tqdm(gen_queries):
#     resp = get_response_rag(q, is_poem=True)
#     gen_response_rag_lst.append(resp)

# gen_response_rag_lst2 = []
# for q in tqdm(gen_queries):
#     resp = get_response_rag(q, is_poem=True)
#     gen_response_rag_lst2.append(resp)

gen_response_rag_lst3 = []
for q in tqdm(gen_queries):
    resp = get_response_rag(q, is_poem=True)
    gen_response_rag_lst3.append(resp)

100%|██████████| 5/5 [01:02<00:00, 12.53s/it]


In [ ]:
gen_response_rag_lst

['Зимний день в деревне, где снег лежит,\nИ ветер шепчет, как будто в тишине.\nПришла пора, когда уютный свет\nСвечи мерцает, согревая вьюги звенья.\n\nСлуга принёс мне чашку чая тепла,\nИ я, взглянув в окно, вижу мир чистый,\nГде белый снег, как письмо, ждёт меня,\nИ мысли в даль уносятся, как лебедь в море.\n\n"Ах, зимняя пора, ты как мечта!" —\nШепчу я, вспоминая старинные сказы.\nНа коне встаём, и ветер встречает нас,\nКак старый друг, и снег летит нам в лицо.\n\nМы едем по полю, где сосны стоят,\nИх шорох ветвей, как музыка в ночи.\nДорога вьётся, как струна, между нами,\nИ вдали, как призрак, встаёт луна.\n\n"Вот и тройка мчится," — слышу я вдали,\nКолокольчик звенит, как зов счастливый.\nИ в сердце вновь восстаёт былого стих,\nКак будто голос прошлого в этот миг.\n\nНо вот и дом, и тёплый свет окон,\nИ снова в кругу семьи, как в детстве.\nЗима — она как книга, где страницы —\nМинуты, что мы проводим в тишине.',
 'Вот мой стих, вдохновленный духом Маяковского:\n\n**Патриотическая

In [ ]:
gen_response_rag_lst2

['Зимой, когда в деревне тишь,\nИ снег вьюжит, словно в сказке,\nЯ вспоминаю время, как\nПрирода вдруг оживает.\n\nСлуга мне чашку чая нёс,\nИ в мыслях был я далеко:\n"А может, в поле снег покроет,\nИ можно будет лыжи взять?"\n\nСнежинки танцуют вокруг,\nКак вальсируют белоснежно,\nИ с каждым шагом глубже след,\nКак будто зовет меня в ночь.\n\nВот метель утихла, и солнце\nПробилось сквозь завесу туч.\nСосны стоят, как старцы мудрые,\nИх шорох слышу я, как друг.\n\nНо вдруг, среди зимнего пейзажа,\nРазглядел я юную рощу,\nГде мельница крылья расправила,\nИ тропинка к дому ведет.\n\n"О, зимняя красота!" — вздыхаю,\nИ думаю: "Как бы не уйти\nВ эту тишь, где душа отдыхает,\nГде снег и сосны – в сердце живут."\n\nВот и сейчас, стоя на кладбище,\nСмотрю на даль, где ветер шумит,\nИ слышу в тишине колокольчик,\nЧто зовет меня в мир иной.',
 'В сердце кованом, как сталь,  \nСоветский дух, как огонь, горит.  \nНа баррикадах мы стоим,  \nКак в броне, не зная страха,  \nЗа Родину свою.\n\nПусть в

In [ ]:
gen_response_rag_lst3

['Зимний день в деревне тих и бел,\nСнегопадом покрыта земля,\nИ слуга, с чашкой чая в руке, мне вежливо ведает:\n"Метель утихла, пороша в лесу легла."\n\nВстаём, и кони под уздцы берём,\nПо полю рысью, где вьюга завывает.\nАрапники в руках, собаки за нами следуют,\nСмотрим на снег, и мысли улетают вдаль.\n\nСквозь ветви сосен, как тени, тихо шуршат,\nИ старая тропа под ногами снегом укрыта.\nТам, где дорога в гору поднимается,\nТри сосны стоят, как часовые, молчаливы.\n\nНо вглядись: под ними молодая роща,\nСемейство зелёных ветвей, как дети вокруг.\nА одиноко стоит сосна старая,\nКак ветер в душе, напоминает о былом.\n\nЗима в деревне — время тихих раздумий,\nКогда душа ищет покой в тишине.\nИ в этом уединении находит ответы,\nНа вопросы, что в сердце давно живут.\n\nЗвон колокольчика в утреннем морозе,\nПесня ямщика — о жизни и о том, что было.\nИ в этих звуках — грусть и радость, любовь и утрата,\nКак будто сама зима шепчет нам о вечности.',
 'В просторах страны, где ветер свободы 

In [44]:
import pandas as pd


gen_authors_lst = ['Александр Пушкин', 'Владимир Маяковский', 'Осип Мандельштам', 'Анна Ахматова', 'Иосиф Бродский']
gen_themes_lst = ['Зима в деревне', 'Советский патриотизм', 'Осенний дождливый день', 'Безответная любовь', 'Одиночество в городе']

pd.DataFrame({
    'author': gen_authors_lst,
    'theme': gen_themes_lst,
    'queries': gen_queries,
    # 'LLM_RAG_LORA1': gen_response_rag_lst,
    # 'LLM_RAG_LORA2': gen_response_rag_lst2,
    'LLM_RAG_LORA3': gen_response_rag_lst3,
}).to_csv('../data/ResponsesGen_lrl_3.csv', index=False)
# .to_csv('../data/ResponsesGen_lrl_2.csv', index=False)
# .to_csv('../data/ResponsesGen_lrl_1.csv', index=False)